In [1]:
import pandas as pd
import os.path
import numpy as np
import PIL
from PIL import Image

In [2]:
os.chdir('/Users/KevinBu/Desktop/clemente_lab/CUTIE/')
stat_df = pd.read_csv('scripts/stat_dict_real.txt', sep = '\t')
rev_stat_df = pd.read_csv('scripts/rev_stat_dict_real.txt', sep = '\t')
stat_df.head()

,statistic,lungc,hdac,who
0,pearson,lungc_kpc,hdac_kpc,WHO_kpc
1,spearman,lungc_ksc,hdac_ksc,WHO_ksc
2,kendall,lungc_kkc,hdac_kkc,WHO_kkc
3,mine,lungc_mine,hdac_mine,WHO_mine


In [3]:
def create_img(label, rev_label):
    # define image names
    spacer = np.ones(shape=[480,640,4]) * 255
    for_img_list = ['sample_corr_false_corr_FP_1.png', 'sample_corr_true_corr_TP_1.png']

    rev_img_list = ['sample_corr_false_corr_TN_1.png', 'sample_corr_true_corr_FN_1.png']

    for_dir_path = '../Submissions/CUtIe/final_data_fixed/cutie_' + label + '1fdr0.05/graphs/'
    rev_dir_path = '../Submissions/CUtIe/final_data_fixed/cutie_' + rev_label + '1fdr0.05/graphs/'

    rev_list = [rev_dir_path + x for x in rev_img_list] # TN, FN, TN, FN
    for_list = [for_dir_path + x for x in for_img_list] # FP, TP, FP, TP

    sample_imgs = [for_list[1], for_list[0], rev_list[0], rev_list[1]]

    opened_sample_imgs = []
    for i in sample_imgs:
        try:
            opened_sample_imgs.append(np.asarray(PIL.Image.open(i)))
        except:
            opened_sample_imgs.append(spacer)
            print(i)

    sample_imgs_comb = np.hstack(opened_sample_imgs)
    
    return sample_imgs_comb

In [4]:
statistics = stat_df['statistic'].values

# for each dataset
for dataset in stat_df.columns.values[1:4]:
    # statistic = 'pearson'
    subset_df = stat_df.set_index('statistic').loc[:,dataset]
    rev_subset_df = rev_stat_df.set_index('statistic').loc[:,dataset]
    
    # for each stat

    # for each of the 4 metrics
    # gather all four metrics in order TP, FP, TN, FN
    # do for true and do for sample (separate images)

    # pearson
    sample_pc_imgs_comb = create_img(subset_df.loc['pearson'], rev_subset_df.loc['pearson'])

    # spearman
    sample_sc_imgs_comb = create_img(subset_df.loc['spearman'], rev_subset_df.loc['spearman'])

    # kendall
    sample_kc_imgs_comb = create_img(subset_df.loc['kendall'], rev_subset_df.loc['kendall'])

    # mine 
    sample_mine_imgs_comb = create_img(subset_df.loc['mine'], rev_subset_df.loc['mine'])

    # combine images
    sample_imgs_comb = np.vstack([sample_pc_imgs_comb, sample_sc_imgs_comb, sample_kc_imgs_comb, sample_mine_imgs_comb])

    # insert spacing of same height but 1/8th the width
    imgs_comb = PIL.Image.fromarray(np.uint8(sample_imgs_comb))
    imgs_comb.save('../Submissions/CUtIe/final_data_fixed/plot_dist_real_' + dataset + '.png') 